# Simulation

In [28]:
import numpy as np
import pandas as pd

# Constants in SI units
c = 4186  # Specific heat capacity of water in J/kg·K
L = 2260000  # Latent heat of vaporization for water in J/kg
Kb = 0.512  # Ebullioscopic constant for water in K kg/mol
molar_mass_NaCl = 0.05844  # Molar mass of NaCl in kg/mol
P0 = 101325  # Standard atmospheric pressure in Pa

np.random.seed(42)  # Set random seed for reproducibility

# Simulation parameters
n_samples = 1000000  # Number of samples to generate
TDS_range = (150, 1200)  # Total Dissolved Solids range in g/kg
temperature_range = (0, 273)  # Temperature range in K

# DataFrame to store simulation data
columns = ['TDS', 'Initial_Temperature_K', 'Vapor_Pressure_Pa', 'Molality', 'Final_Temperature_K', 'Energy_Consumed_J']
data = pd.DataFrame(index=range(n_samples), columns=columns)

def calculate_final_temperature(T0, Kb, molality):
    normal_boiling_point = 373.15
    return normal_boiling_point + Kb * molality


def calculate_relative_vapor_pressure(final_temp_K, initial_temp_K):
    """Provides a value reflecting how vapor pressure changes based on temperatures."""
    return np.exp(-1 / (final_temp_K - initial_temp_K))   # Arbitrary function


def calculate_energy(initial_temp_K, final_temp_K, mass=1):
    delta_T = final_temp_K - initial_temp_K
    energy_heating = mass * c * delta_T  # Energy to heat the water to the final temperature
    energy_vaporization = mass * L  # Energy to vaporize the water (if applicable)
    total_energy = energy_heating + energy_vaporization
    return total_energy



# Generate and compute data within the simulation loop
for i in range(n_samples):
    TDS = np.random.uniform(*TDS_range)
    initial_temp_K = np.random.uniform(*temperature_range)

    molality = TDS / (molar_mass_NaCl * 1000)

    final_temp_K = calculate_final_temperature(initial_temp_K, Kb, molality)

    # Consider using a more advanced equation if needed (e.g., Antoine equation)
    vapor_pressure = calculate_relative_vapor_pressure(final_temp_K, initial_temp_K) * P0

    energy_consumed = calculate_energy(initial_temp_K, final_temp_K)

    # Populate the DataFrame
    data.loc[i] = [TDS, initial_temp_K, vapor_pressure, molality, final_temp_K, energy_consumed]

# Save the processed data to a CSV file
data.to_csv('data.csv', index=False)


In [29]:
data

,TDS,Initial_Temperature_K,Vapor_Pressure_Pa,Molality,Final_Temperature_K,Energy_Consumed_J
0,543.267125,259.545006,100472.564715,9.296152,377.90963,2755474.316863
1,918.593639,163.433766,100860.769733,15.718577,381.197911,3171560.711931
2,313.819572,42.586504,101021.461935,5.369945,375.899412,3655247.831512
3,210.987793,236.466088,100596.216205,3.610332,374.99849,2839896.635215
4,781.170762,193.303814,100783.706694,13.367056,379.993933,3041484.838212
...,...,...,...,...,...,...
999995,242.031815,193.307695,100769.682646,4.141544,375.27047,3021696.17793
999996,957.699338,233.49832,100642.873203,16.387737,381.540521,2879704.654971
999997,597.557891,125.738194,100924.739146,10.225152,378.385278,3317580.691928
999998,1100.41849,186.489254,100810.142615,18.829885,382.790901,3081718.695073


# Training

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from scipy.optimize import minimize
import pickle


# Constants
c = 4186  # J/kg·K, specific heat capacity of water
L = 2260000  # J/kg, latent heat of vaporization of water

# Load and preprocess the dataset
print("Loading and preprocessing data...")
data = pd.read_csv('data.csv')

# Clean the data
data.fillna(method='ffill', inplace=True)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)
column_req = ['TDS', 'Initial_Temperature_K', 'Molality', 'Energy_Consumed_J']

# Normalize data using RobustScaler
scaler = RobustScaler()
data[column_req] = scaler.fit_transform(data[column_req])

# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Model training for temperature
print("Training temperature model...")
temp_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
temp_model.fit(train_data[column_req], train_data['Final_Temperature_K'])

# Model training for pressure (assuming 'Vapor_Pressure_Pa' is the target variable)
print("Training pressure model...")
pressure_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
pressure_model.fit(train_data[column_req], train_data['Vapor_Pressure_Pa'])

# Save the trained models and scaler to disk using pickle
with open('temp_model.pkl', 'wb') as f:
    pickle.dump(temp_model, f)
    print("Temperature model saved to temp_model.pkl.")

with open('pressure_model.pkl', 'wb') as f:
    pickle.dump(pressure_model, f)
    print("Pressure model saved to pressure_model.pkl.")

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    print("Scaler saved to scaler.pkl.")

print("Model training and saving completed successfully.")


Loading and preprocessing data...
Training temperature model...
Training pressure model...
Temperature model saved to temp_model.pkl.
Pressure model saved to pressure_model.pkl.
Scaler saved to scaler.pkl.
Model training and saving completed successfully.


# Inference

In [38]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
import pickle

# Load the trained models and scaler from disk
temp_model = pickle.load(open('temp_model.pkl', 'rb'))
pressure_model = pickle.load(open('pressure_model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))

def calculate_molality(TDS, molar_mass_NaCl=58.44):  # molar mass in g/mol for NaCl
    """Calculate molality given Total Dissolved Solids in g/kg."""
    molality = TDS / molar_mass_NaCl
    return molality

def calculate_energy(initial_temp_K, final_temp_K, mass=1, c=4186):
    """Calculate the energy required to change the temperature of a mass of water."""
    delta_T = final_temp_K - initial_temp_K
    energy = mass * c * delta_T  # Energy in Joules
    return energy

def predict_temperature(initial_temp_K, TDS):
    """Predict the final temperature given initial temperature and TDS."""
    molality = calculate_molality(TDS)
    # Simulated final temperature for energy calculation (may need actual model prediction or another method)
    final_temp_K_simulated = initial_temp_K + 10
    energy = calculate_energy(initial_temp_K, final_temp_K_simulated)
    input_data = pd.DataFrame({
        'TDS': [TDS],
        'Initial_Temperature_K': [initial_temp_K],
        'Molality': [molality],
        'Energy_Consumed_J': [energy]
    })
    scaled_input = scaler.transform(input_data)
    return temp_model.predict(scaled_input)[0]

def predict_pressure(initial_temp_K, TDS):
    """Predict the pressure given initial temperature and TDS."""
    molality = calculate_molality(TDS)
    final_temp_K_simulated = initial_temp_K + 10
    energy = calculate_energy(initial_temp_K, final_temp_K_simulated)
    input_data = pd.DataFrame({
        'TDS': [TDS],
        'Initial_Temperature_K': [initial_temp_K],
        'Molality': [molality],
        'Energy_Consumed_J': [energy]
    })
    scaled_input = scaler.transform(input_data)
    return pressure_model.predict(scaled_input)[0]

def main():
    try:
        initial_temp_C = float(input("Enter the initial temperature in Celsius: "))
        TDS = float(input("Enter the Total Dissolved Solids (TDS) in g/kg: "))

        initial_temp_K = initial_temp_C + 273.15
        predicted_temp_K = predict_temperature(initial_temp_K, TDS)
        predicted_temp_C = predicted_temp_K - 273.15
        predicted_pressure = predict_pressure(initial_temp_K, TDS)
        energy_consumed = calculate_energy(initial_temp_K, predicted_temp_K)

        print(f"Predicted final temperature: {predicted_temp_C:.2f} Celsius")
        print(f"Predicted pressure: {predicted_pressure:.2f} Pa")
        print(f"Energy conserved to reach this temperature: {-energy_consumed:.2f} Joules")

    except ValueError:
        print("Invalid input. Please enter a valid number.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Enter the initial temperature in Celsius: 123
Enter the Total Dissolved Solids (TDS) in g/kg: 321
Predicted final temperature: 102.81 Celsius
Predicted pressure: 100334.55 Pa
Energy consumed to reach this temperature: -84504.16 Joules


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
